<a href="https://colab.research.google.com/github/Turing-04/road_classifier_satellite/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install data
!git clone "https://github.com/Turing-04/road_classifier_satellite.git"
 

Cloning into 'road_classifier_satellite'...
remote: Enumerating objects: 12632, done.
remote: Counting objects: 100% (1218/1218), done.
remote: Compressing objects: 100% (1208/1208), done.
remote: Total 12632 (delta 13), reused 1211 (delta 10), pack-reused 11414
Receiving objects: 100% (12632/12632), 2.38 GiB | 59.68 MiB/s, done.
Resolving deltas: 100% (243/243), done.
Checking out files: 100% (2483/2483), done.


In [ ]:
%cd road_classifier_satellite
%ls


/content/road_classifier_satellite
analysis/             helpers.py             models/    test.py      train.py
data_augmentation.py  loss.py                README.md  training/    utilitary/
dataset.py            mask_to_submission.py  test/      train.ipynb  utils.py


In [ ]:
import os
import time
from glob import glob

import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from dataset import DriveDataset
from loss import DiceLoss, DiceBCELoss
import sys
from utils import seeding, create_dir, epoch_time

from models import model_unet
from models import model_resnet
from models import model_cnn2
from models import model_cnn4
from models import model_cnn8
from models import model_cnn16
from models import model_vggunet

In [ ]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0

    model.train()
    for x, y in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss

In [ ]:
""" Read command line arguments """
model_name = sys.argv[1]

""" Setup """
seeding(42)
create_dir("weights")

""" Load dataset """
train_x = sorted(glob("training/images/training/*"))
train_y = sorted(glob("training/groundtruth/training/*"))

valid_x = sorted(glob("training/images/validation/*"))
valid_y = sorted(glob("training/groundtruth/validation/*"))

data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
print(data_str)

""" Hyperparameters """
size = (400, 400)
batch_size = 2
num_epochs = 50
lr = 1e-4

""" Create dataloader """
train_dataset = DriveDataset(train_x, train_y)
valid_dataset = DriveDataset(valid_x, valid_y)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

""" Load the model """
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


###right here, try to use model= models.vgg16() <- FIX INPUT SIZE (with an extra max polling layer maybe or loading the files differently? idk...)
model = model_vggunet.VGGUNET()

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
loss_fn = DiceBCELoss()

checkpoint_path = "weights/checkpoint_" + model_name + ".pth"
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Checkpoint loaded: {checkpoint_path}")

""" Training the model """
best_valid_loss = float("inf")

for epoch in range(num_epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    valid_loss = evaluate(model, valid_loader, loss_fn, device)

    """ Saving the model """
    if valid_loss < best_valid_loss:
        print(f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}")

        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
    data_str += f'\tTrain Loss: {train_loss:.3f}\n'
    data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
    print(data_str)

Dataset Size:
Train: 1000 - Valid: 100

Valid loss improved from inf to 0.6551. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 01 | Epoch Time: 0m 39s
	Train Loss: 0.924
	 Val. Loss: 0.655

Valid loss improved from 0.6551 to 0.5162. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 02 | Epoch Time: 0m 31s
	Train Loss: 0.606
	 Val. Loss: 0.516

Valid loss improved from 0.5162 to 0.4285. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 03 | Epoch Time: 0m 31s
	Train Loss: 0.498
	 Val. Loss: 0.429

Valid loss improved from 0.4285 to 0.3493. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 04 | Epoch Time: 0m 31s
	Train Loss: 0.441
	 Val. Loss: 0.349

Epoch: 05 | Epoch Time: 0m 31s
	Train Loss: 0.397
	 Val. Loss: 0.362

Valid loss improved from 0.3493 to 0.3455. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 06 | Epoch Time: 0m 31s
	Train Loss: 0.382
	 Val. Loss: 0.346

Valid loss improved from 0.3455 to 0.3152. Saving checkpoint: weights/checkpoint_-f.pth
Epoch: 07 | Epo